## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-21-13-00-03 +0000,bbc,How WhatsApp messages revealed ex-Reform polit...,https://www.bbc.com/news/articles/cn8vnv3dk0vo...
1,2025-11-21-13-00-00 +0000,wsj,Are Stocks a Better Bet Than Social Security?,https://www.wsj.com/finance/investing/are-stoc...
2,2025-11-21-13-00-00 +0000,wsj,When Toyota was planning to build its first st...,https://www.wsj.com/arts-culture/books/martha-...
3,2025-11-21-12-58-01 +0000,bbc,Gove apologises after Covid report alleges 'to...,https://www.bbc.com/news/articles/c9qeglxwwx7o...
4,2025-11-21-12-56-18 +0000,bbc,Barber to offer 'pay what you can' haircuts,https://www.bbc.com/news/articles/cn98l3x0jrpo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2321/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
116,trump,45
216,new,27
53,ukraine,13
63,plan,13
433,year,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
254,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...,115
211,2025-11-20-23-31-36 +0000,nypost,Ex-New York Post media columnist Keith J. Kell...,https://nypost.com/2025/11/20/media/ex-new-yor...,89
366,2025-11-20-16-22-10 +0000,bbc,Trump to meet incoming New York Mayor Zohran M...,https://www.bbc.com/news/articles/cvgmgddyjl0o...,86
31,2025-11-21-11-55-46 +0000,nypost,Democratic lawmakers get ‘24-7′ security after...,https://nypost.com/2025/11/21/us-news/democrat...,86
15,2025-11-21-12-29-00 +0000,wsj,Trump Peace Plan Demands Major Concessions Fro...,https://www.wsj.com/world/u-s-peace-plan-for-u...,85


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
254,115,2025-11-20-22-15-38 +0000,cbc,Trump administration touts plan for new oil dr...,https://www.cbc.ca/news/world/trump-administra...
178,64,2025-11-21-01-36-00 +0000,wsj,The Transportation Department said it would pa...,https://www.wsj.com/politics/policy/air-traffi...
351,61,2025-11-20-17-20-47 +0000,nypost,Kamala Harris and Jill Biden come face-to-face...,https://nypost.com/2025/11/20/us-news/kamala-h...
352,58,2025-11-20-17-20-17 +0000,nypost,Rubio casts doubt on pro-Russian Ukraine peac...,https://nypost.com/2025/11/20/world-news/rubio...
234,55,2025-11-20-22-37-38 +0000,latimes,Driver charged with murder in crash that kille...,https://www.latimes.com/california/story/2025-...
371,47,2025-11-20-15-45-00 +0000,wsj,Home sales rose to an eight-month high in Octo...,https://www.wsj.com/economy/housing/home-sales...
31,41,2025-11-21-11-55-46 +0000,nypost,Democratic lawmakers get ‘24-7′ security after...,https://nypost.com/2025/11/21/us-news/democrat...
350,40,2025-11-20-17-26-27 +0000,latimes,Fire marched toward west Altadena hours before...,https://www.latimes.com/california/story/2025-...
200,36,2025-11-21-00-04-21 +0000,latimes,L.A. Olympic organizing committee adds former ...,https://www.latimes.com/sports/story/2025-11-2...
195,36,2025-11-21-00-18-58 +0000,wapo,Federal judge dismisses case against Chicago w...,https://www.washingtonpost.com/nation/2025/11/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
